### Домашнее задание №8.

***Взять датасет из google диска: https://drive.google.com/file/d/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_. 
Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и 
значимо ли это отличие статистически.***

In [1]:
import numpy as np
import pandas as pd
from statsmodels.stats import proportion
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

***Необходимые для решения задачи данные содержатся в файле marketing_campaign.csv. Загрузим файл, посмотрим на типы данных и проверим на наличие пропусков.***

In [2]:
marketing_campaign = pd.read_csv('./marketing_campaign.csv')
marketing_campaign

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True
1,a1001,1/1/18,House Ads,personalization,English,True
2,a1002,1/1/18,House Ads,personalization,English,True
3,a1003,1/1/18,House Ads,personalization,English,True
4,a1004,1/1/18,House Ads,personalization,English,True
...,...,...,...,...,...,...
10032,a11032,1/17/18,Email,control,German,True
10033,a11033,1/17/18,Email,control,German,True
10034,a11034,1/5/18,Instagram,control,German,False
10035,a11035,1/17/18,Email,control,German,True


In [3]:
df = marketing_campaign

In [4]:
df.shape

(10037, 6)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             10037 non-null  object
 1   date_served         10021 non-null  object
 2   marketing_channel   10022 non-null  object
 3   variant             10037 non-null  object
 4   language_displayed  10037 non-null  object
 5   converted           10022 non-null  object
dtypes: object(6)
memory usage: 470.6+ KB


***Информация о разбиении клиентов на группы А (control) и В (personalization) содержится в колонке "variant". Посмотрим на количество данных по группам.***

In [6]:
df.variant.value_counts()

control            5091
personalization    4946
Name: variant, dtype: int64

In [7]:
df.converted.value_counts()

False    8946
True     1076
Name: converted, dtype: int64

***Соотношение данных в группах примерно 50/50. Хороший показатель.***

***Данные о конверсии - пользователь совершил покупку (True) и не совершил (False), содежрится в колонке "converted". Количество данных в "converted" не совпадает с количеством в "variant". Посмотрим в "converted":***

In [8]:
df.converted.unique()

array([True, False, nan], dtype=object)

***Имеется незначительное количество пропущенных данных. В расчетах мы не будем их учитывать.***

***Так как столбец converted - содержит бинарный показатель (True=1, False=0), совершил клиент покупку илb нет, то нашей метрикой будет доля (конверсия). В этом случае в качестве критерия оценки подойдет хи-квадрат (критерий согласия Пирсона). Метод позволяет оценить значимость различий между фактическим (выявленным в результате исследования) количеством исходов и теоретическим количеством, которое можно ожидать в изучаемых группах при справедливости нулевой гипотезы. То есть оценить статистическую значимость различий двух или нескольких относительных показателей (частот, долей). Необходимое условие - группы должны быть независимы - в нашем случае соблюдается.***

***Уровень достоверности возьмем стандартным для такого случая alpha = 95% = 0.95. По таблице Z-оценок значение будет равно 1.96.***

***Сформируем гипотезу: Н0: р1 = р2 (конверсия в группах одинаковая); Н1: р1 != р2 (конверсия в группах отличается) и разница статистически значима.***

***Расчитаем наблюдаемые частоты:***

In [9]:
z_crit_value = 1.96 # соответствует доверительному интервалу в 95%
k1 = df[df['variant']=='control']['converted'].sum()
n1 = df[df['variant']=='control'].shape[0]
k2 = df[df['variant']=='personalization']['converted'].sum()
n2 = df[df['variant']=='personalization'].shape[0]

k1, n1, k2, n2

(371, 5091, 705, 4946)

***Выведем полученные данные в таблицу:***

In [10]:
table = df.pivot_table(values='converted', index='variant', aggfunc=['sum', 'count'])
table

,sum,count
,converted,converted
variant,,
control,371,5076
personalization,705,4946


***Проверим гипотезу, используя критерий хи-квадрат:***

In [11]:
chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]), 
                                                        np.array([n1, n2]))

print(f'Results are chisq={chisq:.3f} pvalue={pvalue:.3f}')

Results are chisq=127.213 pvalue=0.000


***Получили значение pvalue=0.000. Неожиданно... Перепроверим полученные данные с помощью другой библиотеки:***

In [12]:
chi2, p, dof, expected = chi2_contingency(table, correction=False)
p

1.5359317873632312e-26

***В действительности pvalue практически равно 0.***

***Так как значение pvalue ниже выбранного порога 0.05 то гипотеза Н0 отвергается.***

#### ВЫВОД: конверсия в тесте с группой В существенно больше, чем конверсия в тесте с гурппой А и это отличие статистически значимо. 

***Так как наши данные непараметрические, можем попробовать проверить их и на Z-критерий:***

In [13]:
from statsmodels.stats import proportion

z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]), 
                                                 np.array([n1, n2]))

print(f'Results are z_score={z_score:.3f} pvalue={z_pvalue:.3f}')

Results are z_score=-11.279 pvalue=0.000


***Получили аналогичный результат: конверсия в группе "В" значительно больше, чем в "А". Другими словами - различные критерии дают одинаковый результат. Значит ошибки быть не должно.***